# MovieLense

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd

2024-11-17 14:36:11.594788: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-17 14:36:11.596248: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-17 14:36:11.603904: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-17 14:36:11.628765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731850571.670577  167382 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731850571.68

In [3]:
def get_data(ds_name = 'movielens/100k-ratings'):
  def genre_id_to_text(genre_ids):
    return [genres[id] for id in genre_ids]

  ds=  tfds.load(ds_name, split='all')
  df = tfds.as_dataframe(ds)

  for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].apply(
            lambda x: x.decode('utf-8') if isinstance(x, bytes) else x
        )

  genres = ['Action' ,'Adventure' ,'Animation',
        'Children' ,'Comedy' ,'Crime' ,'Documentary' ,'Drama' ,'Fantasy',
        'Film-Noir' ,'Horror' ,'IMAX' ,'Musical' ,'Mystery' ,'Romance' ,'Sci-Fi',
        'Thriller' ,'War' ,'Western', 'unknown']
  df['genres_id'] = df['movie_genres']
  df['movie_genres'] = df['movie_genres'].apply(genre_id_to_text)

  return df


In [4]:
data = get_data()

2024-11-17 14:36:17.006967: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2024-11-17 14:36:17.381460: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2024-11-17 14:37:04.997871: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
data.head()

,bucketized_user_age,movie_genres,movie_id,movie_title,raw_user_age,timestamp,user_gender,user_id,user_occupation_label,user_occupation_text,user_rating,user_zip_code,genres_id
0,45.0,[Drama],357,One Flew Over the Cuckoo's Nest (1975),46.0,879024327,True,138,4,doctor,4.0,53211,[7]
1,25.0,"[Comedy, Romance]",709,Strictly Ballroom (1992),32.0,875654590,True,92,5,entertainment,2.0,80525,"[4, 14]"
2,18.0,[Comedy],412,"Very Brady Sequel, A (1996)",24.0,882075110,True,301,17,student,4.0,55439,[4]
3,50.0,"[Crime, Drama]",56,Pulp Fiction (1994),50.0,883326919,True,60,4,healthcare,4.0,06472,"[5, 7]"
4,50.0,"[Horror, Thriller]",895,Scream 2 (1997),55.0,891409199,True,197,18,technician,3.0,75094,"[10, 16]"


In [9]:
df = pd.DataFrame(data, columns=['movie_id','user_id','user_rating', 'genres_id'])
negative_ratings = df[ df['user_rating'] < 4 ].index 

df.drop(negative_ratings, inplace = True) 

In [10]:
df = df.drop(columns=['user_rating'])

In [11]:
df

,movie_id,user_id,genres_id
0,357,138,[7]
2,412,301,[4]
3,56,60,"[5, 7]"
5,325,601,"[7, 16]"
8,425,916,[4]
...,...,...,...
99993,211,60,"[4, 18]"
99994,433,527,[4]
99995,228,774,"[0, 1, 15]"
99996,333,313,"[13, 16]"


In [12]:
from sklearn.model_selection import train_test_split


In [14]:
config = {
    #embedding computation
    'cleora_n_iter': 5,
    'cleora_dim': 1024,
    
    #dataset preparation
    'train_test_split': 0.2,
    
    'batch_size': 256,
    'test_batch_size': 1000,
    'epochs': [10],
    'alpha': [1e-4],
}

In [15]:
train, test = train_test_split(df, test_size=config['train_test_split'])

In [16]:
train.shape, test.shape

((44300, 3), (11075, 3))

In [17]:
len(train.groupby('user_id'))

942

In [22]:
grouped_train = train.groupby('user_id')
for n, (name, group) in enumerate(grouped_train):
    if name=='1':
        continue
    movies_list = group["movie_id"].tolist()
    genres_list = group["genres_id"].tolist()
    group_elems = list(map(str, movies_list))
    print(movies_list)
    print(genres_list)
    break


['276', '85', '367', '23', '617', '274', '530', '93', '588', '511', '699', '629', '153', '194', '711', '234', '98', '223', '488', '510', '432', '22', '371', '157', '16', '474', '606', '211', '495', '116', '693', '707', '483', '1', '197', '527', '502', '124', '135', '505', '100', '519', '656', '4', '470', '414', '156', '203', '200', '651', '582', '275', '333', '700', '321', '518', '161', '705', '174', '286', '132', '706', '205', '525', '709', '185', '589', '696', '482', '168', '420', '162', '712', '216', '144', '701', '509', '531', '218', '504', '186', '133', '230', '447', '418', '357', '486', '340', '489', '558', '654', '238', '657', '655', '245', '11', '32', '7', '12', '499', '269', '289', '56', '385', '69', '664', '708', '127', '191', '480', '164', '285', '686', '703', '9', '521', '160', '82', '484', '137', '497', '498', '611', '603', '615', '195', '170', '496', '50', '199', '604', '302', '134', '463', '404', '475', '694', '64', '283', '478', '493', '179']
[array([ 7, 14]), array([4]

In [203]:
ml_cleora_input_clique_filename = "../movielens/cleora_input_clique.txt"
ml_cleora_input_star_filename = "../movielens/cleora_input_star.txt"
ml_lp_train_filename = "../movielens/lp_train.txt"
ml_lp_test_filename = "../movielens/lp_test.txt"
output_dir = '../output'

In [ ]:
with open(ml_cleora_input_clique_filename, "w") as f_cleora_clique, open(
    ml_cleora_input_star_filename, "w"
) as f_cleora_star, open(ml_lp_train_filename, "w") as f_train:
    grouped_train = train.groupby("user_id")
    for n, (name, group) in enumerate(grouped_train):
        movies_list = group["movie_id"].tolist()
        group_elems = list(map(str, movies_list))
        f_cleora_clique.write("{} {}\n".format(name, " ".join(group_elems)))
        f_cleora_star.write("{}\t{}\n".format(n, name))
        for elem in group_elems:
            f_train.write("{}\t{}\n".format(name, elem))
            f_cleora_star.write("{}\t{}\n".format(n, elem))

In [ ]:
with open(ml_lp_test_filename, "w") as f_test:
    grouped_test = test.groupby('user_id')
    for name, group in grouped_test:
        movies_list = group['movie_id'].tolist()
        group_elems = list(map(str, movies_list))
        for elem in group_elems:
            f_test.write("{}\t{}\n".format(name, elem))

In [206]:
CLEORA_BINARY = '../src/cleora-v1.2.3-x86_64-unknown-linux-gnu'

In [207]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.utils import shuffle
from tqdm import tqdm
import pandas as pd
from collections import Counter
import random
import os

In [208]:
import subprocess


def columns2output_filename(output_dir, columns):
    columns_split = columns.split()
    if len(columns_split) == 1 and 'reflexive' in columns:
        column_name = columns.split('::')[-1]
        return os.path.join(output_dir, f'emb__{column_name}__{column_name}.out')

    column_names = [i.split('::')[-1] for i in columns_split]
    return os.path.join(output_dir, 'emb__' + '__'.join(column_names) + '.out')


def train_cleora(dim, n_iter, columns, input_filename, output_dir):
    command = [CLEORA_BINARY,
                '--columns', columns,
                '--dimension', str(dim), 
                '-n', str(n_iter), 
                '--input', input_filename, 
                '-o', output_dir]
    subprocess.run(command, check=True)
    return columns2output_filename(output_dir, columns)

In [209]:
%%time
cleora_output_star_filename = train_cleora(config['cleora_dim'], config['cleora_n_iter'], "transient::cluster_id StarNode", ml_cleora_input_star_filename, output_dir)

[2024-11-17T13:08:21Z INFO  cleora] Reading args...
[src/main.rs:222] &config = Configuration {
    produce_entity_occurrence_count: true,
    embeddings_dimension: 1024,
    max_number_of_iteration: 5,
    seed: None,
    prepend_field: false,
    log_every_n: 10000,
    in_memory_embedding_calculation: true,
    input: [
        "../movielens/cleora_input_star.txt",
    ],
    file_type: Tsv,
    output_dir: Some(
        "../output",
    ),
    output_format: TextFile,
    relation_name: "emb",
    columns: [
        Column {
            name: "cluster_id",
            transient: true,
            complex: false,
            reflexive: false,
            ignored: false,
        },
        Column {
            name: "StarNode",
            transient: false,
            complex: false,
            reflexive: false,
            ignored: false,
        },
    ],
}
[2024-11-17T13:08:21Z INFO  cleora] Starting calculation...
[src/pipeline.rs:25] &sparse_matrices = [
    SparseMatrix {
   

[2024-11-17T13:08:21Z INFO  cleora::embedding] Done iter: 0. Dims: 1024, entities: 2379, num data points: 90400.
[2024-11-17T13:08:22Z INFO  cleora::embedding] Done iter: 1. Dims: 1024, entities: 2379, num data points: 90400.
[2024-11-17T13:08:22Z INFO  cleora::embedding] Done iter: 2. Dims: 1024, entities: 2379, num data points: 90400.
[2024-11-17T13:08:22Z INFO  cleora::embedding] Done iter: 3. Dims: 1024, entities: 2379, num data points: 90400.


CPU times: user 8.46 ms, sys: 981 μs, total: 9.45 ms
Wall time: 645 ms


[2024-11-17T13:08:22Z INFO  cleora::embedding] Done iter: 4. Dims: 1024, entities: 2379, num data points: 90400.
[2024-11-17T13:08:22Z INFO  cleora::embedding] Done propagating.
[2024-11-17T13:08:22Z INFO  cleora::embedding] Start saving embeddings.
[2024-11-17T13:08:22Z INFO  cleora::embedding] Done saving embeddings.
[2024-11-17T13:08:22Z INFO  cleora::embedding] Finalizing embeddings calculations!
[2024-11-17T13:08:22Z INFO  cleora] Finished in 0 sec


In [210]:
%%time
cleora_output_clique_filename = train_cleora(config['cleora_dim'], config['cleora_n_iter'], "complex::reflexive::CliqueNode", ml_cleora_input_clique_filename, output_dir)

[2024-11-17T13:08:22Z INFO  cleora] Reading args...
[src/main.rs:222] &config = Configuration {
    produce_entity_occurrence_count: true,
    embeddings_dimension: 1024,
    max_number_of_iteration: 5,
    seed: None,
    prepend_field: false,
    log_every_n: 10000,
    in_memory_embedding_calculation: true,
    input: [
        "../movielens/cleora_input_clique.txt",
    ],
    file_type: Tsv,
    output_dir: Some(
        "../output",
    ),
    output_format: TextFile,
    relation_name: "emb",
    columns: [
        Column {
            name: "CliqueNode",
            transient: false,
            complex: true,
            reflexive: true,
            ignored: false,
        },
    ],
}
[2024-11-17T13:08:22Z INFO  cleora] Starting calculation...
[src/pipeline.rs:25] &sparse_matrices = [
    SparseMatrix {
        col_a_id: 0,
        col_a_name: "CliqueNode",
        col_b_id: 1,
        col_b_name: "CliqueNode",
        edge_count: 0,
        hash_2_id: {},
        id_2_hash: [

[2024-11-17T13:08:24Z INFO  cleora::sparse_matrix] Number of entities: 1437
[2024-11-17T13:08:24Z INFO  cleora::sparse_matrix] Number of edges: 3974444
[2024-11-17T13:08:24Z INFO  cleora::sparse_matrix] Number of entries: 797125
[2024-11-17T13:08:24Z INFO  cleora::sparse_matrix] Total memory usage by the struct ~ 22 MB
[2024-11-17T13:08:24Z INFO  cleora] Finished Sparse Matrices calculation in 2 sec
[2024-11-17T13:08:24Z INFO  cleora::embedding] Start initialization. Dims: 1024, entities: 1437.
[2024-11-17T13:08:24Z INFO  cleora::embedding] Done initializing. Dims: 1024, entities: 1437.
[2024-11-17T13:08:24Z INFO  cleora::embedding] Start propagating. Number of iterations: 5.
[2024-11-17T13:08:25Z INFO  cleora::embedding] Done iter: 0. Dims: 1024, entities: 1437, num data points: 797125.
[2024-11-17T13:08:25Z INFO  cleora::embedding] Done iter: 1. Dims: 1024, entities: 1437, num data points: 797125.
[2024-11-17T13:08:25Z INFO  cleora::embedding] Done iter: 2. Dims: 1024, entities: 1437

CPU times: user 11.2 ms, sys: 3.01 ms, total: 14.2 ms
Wall time: 4 s


[2024-11-17T13:08:26Z INFO  cleora::embedding] Done iter: 4. Dims: 1024, entities: 1437, num data points: 797125.
[2024-11-17T13:08:26Z INFO  cleora::embedding] Done propagating.
[2024-11-17T13:08:26Z INFO  cleora::embedding] Start saving embeddings.
[2024-11-17T13:08:26Z INFO  cleora::embedding] Done saving embeddings.
[2024-11-17T13:08:26Z INFO  cleora::embedding] Finalizing embeddings calculations!
[2024-11-17T13:08:26Z INFO  cleora] Finished in 3 sec


In [211]:
def read_embeddings(input_file):
    df_full = pd.read_csv(input_file, delimiter = " ", skiprows=[0], header=None, 
                     index_col=0)
    df_full = df_full.drop([1], axis=1)  
    return df_full

In [212]:
def read_train_test(embeddings):
    valid_idx = embeddings.index.to_numpy()
    train = np.loadtxt(ml_lp_train_filename, delimiter="\t", dtype=np.int32)
    test = np.loadtxt(ml_lp_test_filename, delimiter="\t", dtype=np.int32)
    
    #valid pairs of nodes
    train = train[np.isin(train[:,0], valid_idx) & np.isin(train[:,1], valid_idx)]
    test = test[np.isin(test[:,0], valid_idx) & np.isin(test[:,1], valid_idx)]
    
    #negatives for testset: top 100 most common nodes
    all_idx = train.flatten()
    ctr = Counter(all_idx)
    negatives = ctr.most_common(100)
    negatives = [ n[0] for n in negatives ]
    
    adjacency_dict = dict()
    for inp, out in np.vstack([test, train]):
        if inp not in adjacency_dict:
            adjacency_dict[inp] = set()
        adjacency_dict[inp].add(out)

    return train, test, negatives, adjacency_dict, valid_idx

In [213]:
batch_size = config['batch_size']
test_batch_size = config['test_batch_size']

In [214]:
for algo in [cleora_output_star_filename, cleora_output_clique_filename]:
    embeddings = read_embeddings(algo)
    train_1, test_1, negatives, adjacency_dict, valid_idx = read_train_test(embeddings)
    #for faster operation, draw only 1000 test examples
    test_ex = random.sample(list(test_1), 1000)
    
    #these are the 10.000 most common nodes selected as negatives for each valid testing node pair
    df_neg = embeddings.loc[negatives]
    neg_ids = set(df_neg.index)

    epoch = max(config['epochs'])
    for a in config['alpha']:
        #create a binary classifier outputting whether a node pair represents a valid edge (1) or not a valid edge (0)
        clf = SGDClassifier(random_state=0, loss='log_loss', alpha=a)
        for e in range(0, epoch):
            np.random.shuffle(train_1)
            
            for idx in tqdm(range(0,train_1.shape[0],batch_size)):
                #ones = real pairs of nodes
                #zeros = fake pairs of nodesF
                ones=train_1[idx:min(idx+batch_size,train_1.shape[0]),:]
                
                ones_emb_in = embeddings.loc[ones[:,0]].to_numpy()
                ones_emb_out = embeddings.loc[ones[:,1]].to_numpy()
                #Hadamard
                ones = np.multiply(ones_emb_in,ones_emb_out)
                
                id_train_0_in = np.random.choice(valid_idx, size=len(ones), replace=True)
                id_train_0_out = np.random.choice(valid_idx, size=len(ones), replace=True)
    
                zeros_emb_in = embeddings.loc[id_train_0_in].to_numpy()
                zeros_emb_out = embeddings.loc[id_train_0_out].to_numpy()
                #Hadamard
                zeros = np.multiply(zeros_emb_in, zeros_emb_out)
    
                x_train = np.vstack([ones, zeros])
                y_train = [1]*len(ones) + [0]*len(ones)

                clf.partial_fit(x_train, y_train, classes=[0,1])

            if e+1 in config['epochs']:
                mrr = 0.0
                hr = 0.0
                for n, ex in enumerate(test_ex):
                    l = ex[0]
                    r = ex[1]

                    emb_l = embeddings.loc[l].to_numpy().reshape([1, -1])
                    emb_r = np.vstack((df_neg.to_numpy(), embeddings.loc[r].to_numpy()))
        
                    full_ex = np.hstack([np.repeat(emb_l, len(emb_r), axis=0), emb_r])
                    hadamard = np.multiply(emb_l, emb_r)
                    preds = clf.predict_proba(hadamard)[:,1]
                    preds = np.array(preds)

                    #do not punish for high scores of items from trainset and others from testset
                    forbidden_ex = adjacency_dict[l]
                    df_mask = [0 if (elem in forbidden_ex) else 1 for elem in neg_ids]
                    #last elem is always valid
                    df_mask.append(1)
                    preds *= df_mask
            
                    ranking = (-preds).argsort()
                    rank = np.isin(ranking, 100).nonzero()[0][0]+1
                    mrr += 1/rank
                    hr += (rank <= 10)
                    
                    if (n+1)%100 == 0:
                        print('mrr ', mrr/(n+1), ' hr@10 ', hr/(n+1))

                print('algo: {} epochs: {} lr: {}, mrr: {}, hr@10: {}'.format(algo, str(e+1), a, mrr/len(test_ex), hr/len(test_ex)))

100%|██████████| 174/174 [00:04<00:00, 36.90it/s]


mrr  0.0807419997265284  hr@10  0.2
mrr  0.07374315357420631  hr@10  0.16
mrr  0.07504725078265506  hr@10  0.16333333333333333
mrr  0.07315545529742512  hr@10  0.1625
mrr  0.06822605508844612  hr@10  0.15
mrr  0.06611324227650382  hr@10  0.14833333333333334
mrr  0.06713274933634863  hr@10  0.15142857142857144
mrr  0.06623069411499491  hr@10  0.1425
mrr  0.06716241554067912  hr@10  0.1511111111111111
mrr  0.07087506149066014  hr@10  0.156
algo: ../output/emb__cluster_id__StarNode.out epochs: 10 lr: 0.0001, mrr: 0.07087506149066014, hr@10: 0.156


100%|██████████| 174/174 [00:04<00:00, 35.37it/s]


mrr  0.09454255611798854  hr@10  0.15
mrr  0.09518866356793691  hr@10  0.145
mrr  0.08608859032774639  hr@10  0.13333333333333333
mrr  0.08772875922169014  hr@10  0.1475
mrr  0.08198829148815172  hr@10  0.14
mrr  0.07984310307964018  hr@10  0.14166666666666666
mrr  0.08364773253502399  hr@10  0.14285714285714285
mrr  0.08349941428469224  hr@10  0.1425
mrr  0.08486265976302108  hr@10  0.1388888888888889
mrr  0.0869713273056774  hr@10  0.143
algo: ../output/emb__CliqueNode__CliqueNode.out epochs: 10 lr: 0.0001, mrr: 0.0869713273056774, hr@10: 0.143


In [215]:
test.shape

(11075, 2)